# Temporal SFT Training (Colab)

This notebook runs the full pipeline:
1. Install dependencies
2. Clone repo
3. Upload `train.chat.jsonl` and `val.chat.jsonl`
4. Train LoRA/QLoRA adapter
5. Merge adapter
6. Convert to GGUF
7. Download GGUF

In [ ]:
!nvidia-smi

In [ ]:
!pip -q install -U transformers trl datasets peft accelerate bitsandbytes sentencepiece

In [ ]:
import os
from google.colab import files

In [ ]:
%cd /content
!rm -rf activityfinder-backend
!git clone https://github.com/mihir97/activityfinder-backend.git
%cd /content/activityfinder-backend

Upload the two prepared files from your local repo:
- `data/temporal/sft/train.chat.jsonl`
- `data/temporal/sft/val.chat.jsonl`

In [ ]:
uploaded = files.upload()

In [ ]:
!mkdir -p /content/data
!cp -f /content/train.chat.jsonl /content/data/train.chat.jsonl
!cp -f /content/val.chat.jsonl /content/data/val.chat.jsonl
!ls -lh /content/data

In [ ]:
BASE_MODEL = "Qwen/Qwen2.5-3B-Instruct"
TRAIN_FILE = "/content/data/train.chat.jsonl"
VAL_FILE = "/content/data/val.chat.jsonl"
ADAPTER_DIR = "/content/temporal-qwen25-lora"
MERGED_DIR = "/content/temporal-qwen25-merged"
OUT_DIR = "/content/model-out"
GGUF_BASENAME = "temporal-qwen25"
QUANT_TYPE = "Q4_K_M"

In [ ]:
!python scripts/colab/train_temporal_sft.py \
  --base-model "$BASE_MODEL" \
  --train-file "$TRAIN_FILE" \
  --val-file "$VAL_FILE" \
  --output-dir "$ADAPTER_DIR" \
  --use-4bit \
  --epochs 8 \
  --batch-size 2 \
  --grad-accum 8

In [ ]:
!python scripts/colab/merge_temporal_adapter.py \
  --base-model "$BASE_MODEL" \
  --adapter-dir "$ADAPTER_DIR" \
  --output-dir "$MERGED_DIR"

In [ ]:
%cd /content
!rm -rf llama.cpp
!git clone https://github.com/ggerganov/llama.cpp.git
%cd /content/llama.cpp
!cmake -B build
!cmake --build build --config Release -j

In [ ]:
%cd /content/activityfinder-backend
!LLAMA_CPP_DIR=/content/llama.cpp \
HF_MODEL_DIR="$MERGED_DIR" \
OUT_DIR="$OUT_DIR" \
GGUF_BASENAME="$GGUF_BASENAME" \
QUANT_TYPE="$QUANT_TYPE" \
bash scripts/colab/convert_to_gguf.sh

In [ ]:
!ls -lh "$OUT_DIR"

In [ ]:
import glob
ggufs = sorted(glob.glob(f"{OUT_DIR}/*.gguf"))
print("GGUF files:", ggufs)

In [ ]:
# Download the quantized GGUF (usually ends with q4_k_m.gguf)
target = [x for x in ggufs if "q4_k_m" in x.lower()]
if not target:
    target = ggufs
if not target:
    raise RuntimeError("No GGUF file found")
files.download(target[0])

After download, on your local machine run:

```powershell
.\scripts\importTemporalModel.ps1 -GgufPath C:\\path\\to\\temporal-qwen25.q4_k_m.gguf -ModelName qwen-temporal
npm run predict:temporal-test -- --model qwen-temporal
npm run eval:temporal-test
```